In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [2]:
from llava.mm_utils import get_model_name_from_path, tokenizer_image_token
from llava.model.builder import load_pretrained_model
from llava.constants import IMAGE_TOKEN_INDEX
from llava.eval.run_llava import eval_model
from PIL import Image
import torch

[2024-07-15 15:04:17,716] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
model_path = "../models/llava-v1.6-vicuna-7b-VD/checkpoint-12000"
device = 'cuda:0'

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path)
)

In [10]:
title = "crowded bar"
prompt = f"""USER: You are an AI model designed to help visually impaired people. Your task is to provide a comprehensive description of the image, locating important objects to guide disabled people through the scene.\nBelow is a {title} scene. Provide a descriptive paragraph using a human-like description. Do not add any comment or guess. Remain factual and avoid unnecessary embellishments, keep it simple.\n<image> ASSISTANT:"""
image_file = '../data/image_examples/bar.jpeg'

raw_image = Image.open(image_file).convert('RGB')
image_tensor = image_processor(raw_image, return_tensors='pt')['pixel_values'].to(device).to(torch.float16)
input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(device)

with torch.inference_mode():
    output_ids = model.generate(
                        input_ids,
                        images=image_tensor,
                        do_sample=False,
                        #temperature=0.2,
                        max_new_tokens=500,
                        )

outputs = tokenizer.decode(output_ids[0,:], skip_special_tokens=True).strip()
print(outputs)

Here is a descriptive paragraph of the crowded bar scene:

You are standing in a bustling bar with a long counter stretching across the room. The counter is lined with bottles and glasses, and there are several menus displayed on it. Above the counter, there are three light fixtures hanging from the ceiling. To your left, you see a group of people gathered around the counter, while to your right, there's another group of people seated at tables. The tables are scattered throughout the room, with some near the walls and others in the center of the room. You notice several menus placed on the tables, likely belonging to the patrons. There are also several glasses and bottles scattered around the room, some on the counter and others on the tables. The atmosphere is lively, with many people socializing and enjoying their drinks.
